# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

#os import
path_name = os.path.join("/Users/Vinz/desktop/python-api-challenge/WeatherPy/RandomCities.csv")


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
# Output File (CSV)
#output_data_file1 = "WeatherPy/Randomcities.csv"
gmaps_test = pd.read_csv(path_name)
gmaps_test.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,amga,97,RU,1586736641,90,60.90,131.96,28.96,8.61
1,1,rikitea,45,PF,1586736641,78,-23.12,-134.97,77.95,17.87
2,2,hilo,1,US,1586736642,65,19.73,-155.09,82.40,11.41
3,3,tasiilaq,100,GL,1586736642,97,65.61,-37.64,32.00,40.26
4,4,mataura,73,NZ,1586736643,88,-46.19,168.86,42.01,8.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [89]:
#configure gmaps
gmaps.configure(api_key=g_key)
locations = gmaps_test[["Lat", "Lng"]].astype(float)

#census_data_complete = census_data_complete.dropna()
humidity_test = gmaps_test["Humidity"].astype(float)

#heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_test, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [86]:
#(layout=FigureLayout(height='420px'))


SyntaxError: invalid syntax (<ipython-input-86-87bb64dc1315>, line 1)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = gmaps_test.loc[(gmaps_test["Max Temp"]<80) &
                           (gmaps_test["Max Temp"]>70)
                          & (gmaps_test["Wind Speed"]<10) 
                          & (gmaps_test["Cloudiness"]==0)]
ideal_df

#clean_df = ideal_df.drop("Unnamed:0")

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
28,28,santa cruz,0,BO,1586736655,61,-17.80,-63.17,78.80,6.93
86,86,byron bay,0,AU,1586736477,68,-28.65,153.62,73.40,6.93
108,108,yulara,0,AU,1586736699,35,-25.24,130.99,73.40,8.05
241,241,thohoyandou,0,ZA,1586736807,31,-22.95,30.48,71.47,3.15
370,370,pabna,0,BD,1586736890,81,24.00,89.25,75.42,5.57
401,401,mulege,0,MX,1586736917,46,26.88,-111.98,70.50,2.51
467,467,port macquarie,0,AU,1586736936,67,-31.43,152.92,72.00,4.70
507,507,moctezuma,0,MX,1586736986,18,29.80,-109.67,73.98,9.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
hotel_df = ideal_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
28,28,santa cruz,0,BO,1586736655,61,-17.80,-63.17,78.80,6.93,
86,86,byron bay,0,AU,1586736477,68,-28.65,153.62,73.40,6.93,
108,108,yulara,0,AU,1586736699,35,-25.24,130.99,73.40,8.05,
241,241,thohoyandou,0,ZA,1586736807,31,-22.95,30.48,71.47,3.15,
370,370,pabna,0,BD,1586736890,81,24.00,89.25,75.42,5.57,
401,401,mulege,0,MX,1586736917,46,26.88,-111.98,70.50,2.51,
467,467,port macquarie,0,AU,1586736936,67,-31.43,152.92,72.00,4.70,
507,507,moctezuma,0,MX,1586736986,18,29.80,-109.67,73.98,9.80,


In [92]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"rankby": "distance",
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Lng"]
    params["location"] = f"{lat},{long}"
    
     #print(f"Retrieving Results for Index nearest hotel.")
    response = requests.get(base_url, params=params).json()
    response
    
    results = response['results'][0]
    results

    hotel_df.loc[index,'Hotel Name'] = results["name"]
    
hotel_df
#print(json.dumps(response, indent=4, sort_keys=True))


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
28,28,santa cruz,0,BO,1586736655,61,-17.80,-63.17,78.80,6.93,Hotel Cortez
86,86,byron bay,0,AU,1586736477,68,-28.65,153.62,73.40,6.93,Backpackers Inn on the Beach
108,108,yulara,0,AU,1586736699,35,-25.24,130.99,73.40,8.05,Desert Gardens Hotel - Ayers Rock Resort
241,241,thohoyandou,0,ZA,1586736807,31,-22.95,30.48,71.47,3.15,Masili Guesthouse & Conference
370,370,pabna,0,BD,1586736890,81,24.00,89.25,75.42,5.57,Hotel Dream Palace International
401,401,mulege,0,MX,1586736917,46,26.88,-111.98,70.50,2.51,Casa Granada
467,467,port macquarie,0,AU,1586736936,67,-31.43,152.92,72.00,4.70,Rydges Port Macquarie
507,507,moctezuma,0,MX,1586736986,18,29.80,-109.67,73.98,9.80,Hotel Lobohombo


In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [103]:
#configure gmaps
gmaps.configure(api_key=g_key)
marker_locations = hotel_df[['Lat', 'Lng']]
locations = gmaps_test[["Lat", "Lng"]].astype(float)

#census_data_complete = census_data_complete.dropna()
humidity_test = gmaps_test["Humidity"].astype(float)

#heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_test, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 1)
markers = gmaps.marker_layer(marker_locations)#info_box_contents=[f"Name: {name}" for name in naming_conv])

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
naming_conv = hotel_df["Hotel Name"].tolist()

